In [29]:
import os

In [30]:
%pwd

'/home/atom/project/ENE_mlops_mlflow'

we want to use this code in our whole code that's why we need to change it's dirc

In [31]:
os.chdir("../")

In [32]:
%pwd

'/home/atom/project'

In [33]:
from dataclasses import dataclass
from pathlib import Path
# this is not python class but dataclass. it not use "self"

@dataclass(frozen=True)
class DataIngestionConfig:
    # same as config/config.yaml: that's were we are getting our data
    # Return:   DataIngestionConfig data type
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzipe_dir: Path

In [34]:
from src.mlProject.constant import *
from src.mlProject.utils.common import read_yaml, create_directories

In [35]:
# updating the configuration 

class ConfigurationManger:

    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
# this below code will create artifact of those ^^ files
        create_directories([self.config.artifacts_root])
        
# this one only tells path 
    def get_data_ingestion_config(self)->  DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzipe_dir=config.unzip_dir)
        
        return data_ingestion_config
        

In [36]:
# this code is all about "how it will ingest data"
import os 
import urllib.request as request  # request to download data from URL
import zipfile  # to unzip files
from src.mlProject import logger   # to take logg
from src.mlProject.utils.common import get_size

In [37]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    #below code will donwload data and create file "DataIngestion"
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, 
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded ! with the following info \n{headers}")
        else:
            logger.info(f"file already exists of size {get_size(path(self.config.local_data_file))}")

    # After downloading data we also need to extract it
    def extract_zip_file(self):
        unzip_path = self.config.unzipe_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [38]:
# Now comes best part. this part will hold Pipeline
try:
    config = ConfigurationManger()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config.yaml'